POS Filter

Fitur 132a
Token phrase ditambahkan pre token pembentuknya (pembeda dg 131a-colabs)
- Fitur 1_phrase_detection = yes, Phrase detection menggunakan nlp-id
- Fitur 2_edge_weight = yes, Cooccurences matrix + cosinus similarity (fasttext 200)
- Fitur 3_POS filter = yes
- fitur 4_posisi_kata = yes, judul * 2
- fitur 5_phrase_score = yes, bigram = 2x, trigram = 3x
- fitur 6_combined_TFIDF = no

In [26]:
#1. rutin1 import module
import pandas as pd
import os
import sys
import warnings
import matplotlib.pyplot as plt
import numpy as np
import re
import string

import networkx as nx
from collections import defaultdict
from gensim.models import Word2Vec
#from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

warnings.simplefilter(action='ignore', category=UserWarning)

In [27]:
#2. rutin2 membuat syspath ke root utk aktifkan __init__.py
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
sys.path.append(repo_root)

#3. rutin3 Load the dataset
dataset_path = os.path.join(repo_root, "data/dataset_ekstraksi_r30_lg.xlsx")
df = pd.read_excel(dataset_path)
df["text"] = df["judul"] +". "+ df["isi"]

# SETUP & PREPROCESS

In [28]:
#3 Preprocess
'''
stopwords tidak masuk dalam preprocessing
'''
def preprocess(text):
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    text = text.replace('.', '. ')
    text = re.sub('[^a-zA-Z.]', ' ', text)
    text = text.lower()
    text = re.sub("(\\d|\\W)+"," ",text)
    text = text.strip()

    return text

df["text"] = df['text'].apply(preprocess)
df["judul"] = df["judul"].apply(preprocess)

In [29]:
#4 Load stopword
def load_stopwords(repo_root, stopwords_file_path):
    """
    Load stopwords from a file located in the repository.

    :param repo_root: Root directory of the repository.
    :param stopwords_file_path: Relative path to the stopwords file from the repo_root.
    :return: Set of stopwords.
    """
    stopwords_path = os.path.join(repo_root, stopwords_file_path)
    with open(stopwords_path, 'r') as file:
        stopwords = set(file.read().strip().splitlines())
    return stopwords

repo_root = repo_root
stopwords = load_stopwords(repo_root, "notebooks/stopwords_tuning/all_stop_words.txt")

# FITUR-1 Phrase Detection

Dilakukan sebelum komputasi TextRank
detect_all_tokens
|--incorporate_bigrams_trigrams(unigrams, bigrams, trigrams)
   |--detect_bigram(text):
   |--detect_trigram(text):

ada 3 pilihan
1. frase menggantikan token pembentuknya
2. frase ditambahkan pada awal token pembentuk pertama
3. frase ditambahkan pada akhir token pembentuk terakhir
4. frase ditambahkan 2x pre+post token pembentuk terakhir

Pemilihannya see script dibawah

In [30]:
# FITUR-1 Fungsi Phrase Detection

#from collections import Counter
#from nltk.util import ngrams
#from nlp_id_local.tokenizer import PhraseTokenizer 
#from nlp_id_local.postag import PosTag

from nlp_id.tokenizer import PhraseTokenizer 
from nlp_id.postag import PosTag

def detect_bigram(text):
    
    tokenizer = PhraseTokenizer()
    phrases = tokenizer.tokenize(text)
    # Include only bigrams whose individual words are in available_tokens
    bigrams_only = [phrase for phrase in phrases if phrase.count(" ") == 1]

    return bigrams_only

def detect_trigram(text):

    tokenizer = PhraseTokenizer()
    phrases = tokenizer.tokenize(text)
    # Include only trigrams whose individual words are in available_tokens
    trigrams_only = [phrase for phrase in phrases if phrase.count(" ") == 2 ]

    return trigrams_only

"""
# Phrase replace constructed Token
def incorporate_bigrams_trigrams(unigrams, bigrams, trigrams):
    combined_tokens = []
    skip = 0

    for i in range(len(unigrams)):
        if skip > 0:
            skip -= 1
            continue

        bigram_formed = ' '.join(unigrams[i:i+2]) in bigrams
        trigram_formed = ' '.join(unigrams[i:i+3]) in trigrams

        if bigram_formed:
            combined_tokens.append(' '.join(unigrams[i:i+2]))
            skip = 1
        elif trigram_formed:
            combined_tokens.append(' '.join(unigrams[i:i+3]))
            skip = 2
        else:
            combined_tokens.append(unigrams[i])

    return combined_tokens
"""

# Phrase add pre-constructed Token
def incorporate_bigrams_trigrams(unigrams, bigrams, trigrams):
    combined_tokens = []

    for i in range(len(unigrams)):
        bigram_formed = ' '.join(unigrams[i:i+2]) in bigrams
        trigram_formed = ' '.join(unigrams[i:i+3]) in trigrams

        if trigram_formed:
            combined_tokens.append(' '.join(unigrams[i:i+3]))
        elif bigram_formed:
            combined_tokens.append(' '.join(unigrams[i:i+2]))

        combined_tokens.append(unigrams[i])

    return combined_tokens

"""
# Phrase add twice pre/post-constructed Token
def incorporate_bigrams_trigrams(unigrams, bigrams, trigrams):
    combined_tokens = []

    for i in range(len(unigrams)):
        bigram_formed = ' '.join(unigrams[i:i+2]) in bigrams
        trigram_formed = ' '.join(unigrams[i:i+3]) in trigrams

        if trigram_formed:
            phrase = ' '.join(unigrams[i:i+3])
            combined_tokens.append(phrase)  # Append before the unigrams
            combined_tokens.extend(unigrams[i:i+3])
            combined_tokens.append(phrase)  # Append after the unigrams
        elif bigram_formed:
            phrase = ' '.join(unigrams[i:i+2])
            combined_tokens.append(phrase)  # Append before the unigrams
            combined_tokens.extend(unigrams[i:i+2])
            combined_tokens.append(phrase)  # Append after the unigrams
        else:
            combined_tokens.append(unigrams[i])

    return combined_tokens
"""

def detect_all_tokens(text):
    unigrams = [word for word in text.split()]
    bigrams = detect_bigram(text)
    trigrams = detect_trigram(text)
    
    # Incorporating bigrams and trigrams into the sequence of tokens
    all_tokens = incorporate_bigrams_trigrams(unigrams, bigrams, trigrams)

    return all_tokens

# example
text = df["text"][1]
words = detect_all_tokens(text)
print(text)
print("================================================================")
print(words)

template document jtb gpf project mengacu kepada dokumen jtb cp ctr exhibit coordination procedure kami sampaikan template document yang akan dipergunakan pada proyek jambaran tiung biru jtb gas processing facilities gpf demikian disampaikan sebagai acuan pengelolaan dokumen atas perhatiannya kami ucapkan terima kasih
['template document', 'template', 'document', 'jtb', 'gpf', 'project', 'mengacu', 'kepada', 'dokumen', 'jtb', 'cp', 'ctr', 'exhibit', 'coordination', 'procedure', 'kami', 'sampaikan', 'template document', 'template', 'document', 'yang', 'akan', 'dipergunakan', 'pada', 'proyek', 'jambaran', 'tiung', 'biru', 'jtb', 'gas', 'processing', 'facilities', 'gpf', 'demikian', 'disampaikan', 'sebagai', 'acuan', 'pengelolaan', 'dokumen', 'atas', 'perhatiannya', 'kami', 'ucapkan', 'terima', 'kasih']


WORD EMBEDDING SELECTION
ada 2 pilihan dalam vector soze sama = 200
1. word2vec
2. fasttext

pilihan see script below

In [ ]:
'''
# Load Word2vec Model
w2v_model_path = os.path.join(repo_root, "models/embedding_model/w2v_wiki_own_phrase_training_200.model")

def load_word2vec_model(model_path):
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"The provided Word2Vec model path does not exist: {model_path}")
    
    w2v_model = Word2Vec.load(model_path) 
    available_tokens = set(w2v_model.wv.key_to_index)
    
    return w2v_model, available_tokens
    
w2v_model, available_tokens = load_word2vec_model(w2v_model_path)

# Function to compose vector for a phrase if not defined on the w2v model
def get_phrase_embedding(phrase, w2v_model):
    """Get the averaged word embedding for a phrase."""
    words = phrase.split()
    embeddings = [w2v_model.wv[word] for word in words if word in w2v_model.wv.key_to_index]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return None

# Example
phrase = "bekerja sama"
phrase_vector = get_phrase_embedding(phrase, w2v_model)
phrase_vector

# Cosine similarity word2vec
def get_cosine_similarity(w1, w2, w2v_model):
    vec1 = get_phrase_embedding(w1, w2v_model)
    vec2 = get_phrase_embedding(w2, w2v_model)

    if vec1 is None or vec2 is None:
        return 0

    similarity = cosine_similarity([vec1], [vec2])[0][0]
    return similarity

# example, seharusnya dekat hubungan 2 vector ini
w1 = 'gotong royong'
w2 = 'bekerja sama'
w2v_model, available_tokens = load_word2vec_model(w2v_model_path)
cos_sim = get_cosine_similarity(w1, w2, w2v_model)
cos_sim

'''

# Path to the FastText model
fasttext_model_path = os.path.join(repo_root, "models/embedding_model/fasttext_ina_200_with_phrases.bin")
from gensim.models.fasttext import load_facebook_model

def load_fasttext_model(model_path):
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"The provided FastText model path does not exist: {model_path}")
    
    # Use load_facebook_model for full model loading
    ft_model = load_facebook_model(model_path)
    available_tokens = set(ft_model.wv.key_to_index)
    
    return ft_model, available_tokens
    
ft_model, available_tokens = load_fasttext_model(fasttext_model_path)

# Check the size of the word vectors
vector_size = ft_model.wv.vector_size
print("The size of the FastText word vectors is:", vector_size)

# Function to compose vector for a phrase if not defined in the FastText model
def get_phrase_embedding(phrase, ft_model):
    """Get the averaged word embedding for a phrase."""
    words = phrase.split()
    embeddings = [ft_model.wv[word] for word in words if word in ft_model.wv.key_to_index]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return None
    
# Example usage
phrase = "bekerja sama"
phrase_vector = get_phrase_embedding(phrase, ft_model)
#print(phrase_vector)

# Cosine similarity fasttext
def get_cosine_similarity(w1, w2, ft_model):
    vec1 = get_phrase_embedding(w1, ft_model)
    vec2 = get_phrase_embedding(w2, ft_model)

    if vec1 is None or vec2 is None:
        return 0

    similarity = cosine_similarity([vec1], [vec2])[0][0]
    return similarity

# Example usage with the FastText model
w1 = 'gotong royong'
w2 = 'bekerja sama'
ft_model, available_tokens = load_fasttext_model(fasttext_model_path)
cos_sim = get_cosine_similarity(w1, w2, ft_model)
#print(cos_sim)

# FITUR-2 dan FUNGSI KOMPUTASI TEXTRANK

In [43]:
#text = df["text"][1]
#words = detect_all_tokens(text)

# FITUR-2 dan FUNGSI KOMPUTASI TEXTRANK

# Build Co-occurrence Matrix
def build_co_occurrence_matrix(words, window_size=3):
    co_occurrence = defaultdict(int)
    for i in range(len(words) - window_size + 1):
        window = words[i:i+window_size]
        for j in range(window_size):
            for k in range(j+1, window_size):
                w1, w2 = sorted([window[j], window[k]])
                if w1 != w2:
                    co_occurrence[(w1, w2)] += 1
    return co_occurrence

# Build Graph and Compute TextRank
def build_graph_and_compute_textrank(co_occurrence, embedding_model):
    G = nx.Graph()
    for (w1, w2), weight1 in co_occurrence.items():
        weight2 = get_cosine_similarity(w1, w2, embedding_model)
        weight3 = weight1 * weight2
        if weight2 > 0:
            G.add_edge(w1, w2, weight=weight3)
    return nx.pagerank(G)


# Example
co_occurrence = build_co_occurrence_matrix(words)
#co_occurrence

#scores = build_graph_and_compute_textrank(co_occurrence, w2v_model)
scores = build_graph_and_compute_textrank(co_occurrence, ft_model)

print("mengandung stopword:", scores)

mengandung stopword: {'template': 0.0421080428605898, 'template document': 0.03960151787052916, 'document': 0.04131462711681653, 'jtb': 0.04895191665665787, 'gpf': 0.037328154194784864, 'project': 0.018234573903987993, 'mengacu': 0.018818659884664517, 'kepada': 0.022595764760747386, 'dokumen': 0.036897513323890886, 'cp': 0.022306826832770847, 'ctr': 0.02463487901139442, 'exhibit': 0.022603521806791767, 'coordination': 0.029099841554282316, 'procedure': 0.025660045177046495, 'kami': 0.043783472954031066, 'sampaikan': 0.019721460849721408, 'yang': 0.026942005814030903, 'akan': 0.03484519004679631, 'dipergunakan': 0.028651558606253402, 'pada': 0.024362089789276273, 'proyek': 0.011888497262118821, 'tiung': 0.017959596975130408, 'biru': 0.020135927145690236, 'gas': 0.022290050038132364, 'processing': 0.029938532091744824, 'facilities': 0.02697690664000319, 'demikian': 0.022758168515149793, 'disampaikan': 0.02906134032888613, 'sebagai': 0.030953231538632595, 'acuan': 0.03004362755749552, 'pe

In [59]:
# Cleaning stop word for only unigram
def filter_stopwords(scores, stopwords):
    """
    Filters out stopwords from the scores dictionary only for unigrams. 
    """
    filtered_scores = {}
    for term, score in scores.items():
        words = term.split()
        # Filter out the term if it is a unigram and a stopword
        if len(words) == 1 and words[0] in stopwords:
            continue
        # Include the term otherwise
        filtered_scores[term] = score
    return filtered_scores

scores_wo_stopword = filter_stopwords(scores, stopwords)
scores_wo_stopword

{'template': 0.0421080428605898,
 'template document': 0.03960151787052916,
 'document': 0.04131462711681653,
 'project': 0.018234573903987993,
 'mengacu': 0.018818659884664517,
 'ctr': 0.02463487901139442,
 'exhibit': 0.022603521806791767,
 'coordination': 0.029099841554282316,
 'procedure': 0.025660045177046495,
 'tiung': 0.017959596975130408,
 'biru': 0.020135927145690236,
 'processing': 0.029938532091744824,
 'facilities': 0.02697690664000319,
 'acuan': 0.03004362755749552,
 'pengelolaan': 0.026752032802252843,
 'perhatiannya': 0.02632635305826673}

In [71]:
# FITUR 3_POS_filter

def postag_tokens(tokens):
    postagger = PosTag()
    pos_tags = []

    for token in tokens:
        if ' ' in token:
            # Process the token as a phrase
            phrase_tags = postagger.get_phrase_tag(token)
            # Ensure phrase_tags are in the format [(word, tag), ...]
            pos_tags.extend(phrase_tags)
        else:
            # Process the token as a single word
            word_tag = postagger.get_pos_tag(token)
            # Ensure word_tag is in the format [(word, tag)]
            if word_tag:
                pos_tags.extend(word_tag)

    return pos_tags

def attach_pos_tags_with_scores(scores, pos_tags):
    # Create a dictionary from POS tags list
    pos_dict = {word: tag for word, tag in pos_tags}
    
    # Attach POS tags to keywords with scores
    return [(word, score, pos_dict.get(word, 'UNK')) for word, score in scores.items()]

def filter_by_pos(keywords_with_pos, selected_pos):
    return [item for item in keywords_with_pos if item[2] in selected_pos]

# Example usage

def process_scores_with_pos_tags(scores_wo_stopword, selected_pos_tags):
    # Get POS tags for all unique words in the scores dictionary
    unique_words = list(set([word for word in scores_wo_stopword.keys()]))
    pos_tags = postag_tokens(unique_words)

    # Attach POS tags to the scores
    keywords_with_pos = attach_pos_tags_with_scores(scores_wo_stopword, pos_tags)

    # Filter by Selected POS Tags
    scores_w_F3 = filter_by_pos(keywords_with_pos, selected_pos_tags)
    
    return scores_w_F3

# Example usage
selected_pos_tags = {'NN', 'NNP', 'VB', 'NP', 'VP', 'FW'}
scores_w_F123 = process_scores_with_pos_tags(scores_wo_stopword, selected_pos_tags)
print(scores_w_F123)


[('template', 0.0421080428605898, 'FW'), ('template document', 0.03960151787052916, 'NP'), ('document', 0.04131462711681653, 'NNP'), ('project', 0.018234573903987993, 'NN'), ('mengacu', 0.018818659884664517, 'VB'), ('ctr', 0.02463487901139442, 'NN'), ('exhibit', 0.022603521806791767, 'NN'), ('coordination', 0.029099841554282316, 'FW'), ('procedure', 0.025660045177046495, 'NN'), ('tiung', 0.017959596975130408, 'NN'), ('processing', 0.029938532091744824, 'NN'), ('facilities', 0.02697690664000319, 'NN'), ('acuan', 0.03004362755749552, 'NN'), ('pengelolaan', 0.026752032802252843, 'NN')]


In [72]:
# Fitur 4_posisi_kata = yes, judul * 2
# baru bisa di lakukan setelah textRank

def adjust_scores_for_title(scores_w_F3, title, title_score=2):
    # Split the title into individual words for comparison
    title_words = set(title.split())

    # Create a new list to store adjusted scores
    adjusted_scores = []

    for token, score, pos in scores_w_F3:
        # Check if the token (or any word in a phrase) is in the title
        if any(word in title_words for word in token.lower().split()):
            adjusted_score = score * title_score
        else:
            adjusted_score = score

        # Append the adjusted score tuple to the new list
        adjusted_scores.append((token, adjusted_score, pos))

    return adjusted_scores

# Example Usage
title = df["judul"][1]# Replace with actual title
#scores_w_F3 = [('template', 0.0421080428605898, 'FW'), ('template document', 0.03960151787052916, 'NP'), ...]  # truncated for brevity
scores_w_F1234 = adjust_scores_for_title(scores_w_F123, title)

print("scores_w_F1234 :", scores_w_F1234)


scores_w_F1234 : [('template', 0.0842160857211796, 'FW'), ('template document', 0.07920303574105832, 'NP'), ('document', 0.08262925423363306, 'NNP'), ('project', 0.036469147807975985, 'NN'), ('mengacu', 0.018818659884664517, 'VB'), ('ctr', 0.02463487901139442, 'NN'), ('exhibit', 0.022603521806791767, 'NN'), ('coordination', 0.029099841554282316, 'FW'), ('procedure', 0.025660045177046495, 'NN'), ('tiung', 0.017959596975130408, 'NN'), ('processing', 0.029938532091744824, 'NN'), ('facilities', 0.02697690664000319, 'NN'), ('acuan', 0.03004362755749552, 'NN'), ('pengelolaan', 0.026752032802252843, 'NN')]


In [74]:
def filter_and_rank_keywords(scores_w_F1234, bigram_weight=2, trigram_weight=3):
    adjusted_scores = []

    for token, score, pos in scores_w_F1234:
        token_count = len(token.split())

        # Adjust the score based on the number of words in the token
        if token_count == 2:  # Bigram
            adjusted_score = score * bigram_weight
        elif token_count == 3:  # Trigram
            adjusted_score = score * trigram_weight
        else:  # Unigram or more than 3 words
            adjusted_score = score

        adjusted_scores.append((token, adjusted_score, pos))

    # Sort the words based on adjusted scores
    adjusted_scores.sort(key=lambda x: x[1], reverse=True)

    return adjusted_scores

# Example Usage
scores_w_F12345 = filter_and_rank_keywords(scores_w_F1234, bigram_weight=2, trigram_weight=3)

print("scores_w_F12345 :", scores_w_F12345)


scores_w_F12345 : [('template document', 0.15840607148211663, 'NP'), ('template', 0.0842160857211796, 'FW'), ('document', 0.08262925423363306, 'NNP'), ('project', 0.036469147807975985, 'NN'), ('acuan', 0.03004362755749552, 'NN'), ('processing', 0.029938532091744824, 'NN'), ('coordination', 0.029099841554282316, 'FW'), ('facilities', 0.02697690664000319, 'NN'), ('pengelolaan', 0.026752032802252843, 'NN'), ('procedure', 0.025660045177046495, 'NN'), ('ctr', 0.02463487901139442, 'NN'), ('exhibit', 0.022603521806791767, 'NN'), ('mengacu', 0.018818659884664517, 'VB'), ('tiung', 0.017959596975130408, 'NN')]


In [ ]:
# Fitur 6_combined_TFIDF 
# Compute TFIDF
# Determine document = 
# Determine corpus = 
# determine TFIDF score for only word on previous list = scores_w_F12345

In [36]:
# Fitur 6_combined_TFIDF = no
# baru bisa di lakukan setelah textRank
from sklearn.feature_extraction.text import TfidfVectorizer

def get_ngram_type(token):
    return len(token.split())

def compute_and_adjust_tfidf_scores(words, scores):
    """
    Computes TF-IDF scores for the given words and adjusts the TextRank scores based on n-gram type and TF-IDF scores.

    Parameters:
    - words: List of words from the text.
    - scores: Dictionary of TextRank scores.

    Returns:
    - Adjusted TextRank scores.
    """
    # Compute TF-IDF scores
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([' '.join(words)])
    tfidf_scores = dict(zip(vectorizer.get_feature_names_out(), tfidf_matrix.sum(axis=0).A1))
    
    # Modify scores based on n-gram type and TF-IDF scores
    for token in scores:
        ngram_type = get_ngram_type(token)
        tfidf_score = tfidf_scores.get(token, 1)  # default to 1 if token not in TF-IDF scores
        if ngram_type == 1:  # Unigram
            scores[token] *= tfidf_score
        elif ngram_type == 2:  # Bigram
            scores[token] *= 2 * tfidf_score
        elif ngram_type == 3:  # Trigram
            scores[token] *= 2 * tfidf_score
    
    return scores


MAIN PROCESS ITERATION FOR ALL DATASETS

In [37]:
def format_keyphrases_dataframe(keyphrases_scores):
    """
    Formats the keyphrases and their scores into a structured DataFrame.
    Parameters:
    - keyphrases_scores: List of tuples containing keywords, scores, and POS tags.
    """
    df_keyphrases = pd.DataFrame(keyphrases_scores, columns=['Keyword', 'Score', 'pos'])
    
    # Transpose and reindex the DataFrame to ensure consistent structure
    a = pd.DataFrame(df_keyphrases.Keyword).T.reset_index(drop=True)
    a = a.reindex(columns=range(10), fill_value=0)
    b = pd.DataFrame(df_keyphrases.Score).round(3).T.reset_index(drop=True)
    b = b.reindex(columns=range(10), fill_value=0)
    c = pd.DataFrame(df_keyphrases.pos).T.reset_index(drop=True)
    c = c.reindex(columns=range(10), fill_value=0)

    df_keyphrases = pd.concat([a, b, c], axis=1)

    # Ensure the DataFrame has 30 columns, adding missing ones with zero values
    missing_columns = 30 - df_keyphrases.shape[1]
    for _ in range(missing_columns):
        df_keyphrases[df_keyphrases.shape[1]] = 0

    # Naming the columns
    df_keyphrases.columns = ['key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8', 'key_9', 'key_10',
                             'score_1', 'score_2', 'score_3', 'score_4', 'score_5', 'score_6', 'score_7', 'score_8', 'score_9', 'score_10',
                             'pos_1', 'pos_2', 'pos_3', 'pos_4', 'pos_5', 'pos_6', 'pos_7', 'pos_8', 'pos_9', 'pos_10']

    return df_keyphrases


In [ ]:
predict_textrank = pd.DataFrame()
predict_textrank = pd.DataFrame()
#for i in df.index:
for i in df.loc[2:5].index:    
    print('Processing index', i, end='...! ')
    # Processing Text
    text = df["text"][i]
    words = detect_all_tokens(text)
    # Postagging
    pos_tokens = postag_tokens(words)
    # Building Co-occurrence Matrix
    co_occurrence = build_co_occurrence_matrix(words)
    # Building Graph and Computing TextRank
    #scores = build_graph_and_compute_textrank(co_occurrence, w2v_model)
    scores = build_graph_and_compute_textrank(co_occurrence, ft_model)

    # FITUR-5 phrase_phrase_weight
    # Filtering and Ranking Keywords
    scores_w_F5 = filter_and_rank_keywords(scores, stopwords, num_keywords=10, bigram_weight=2, trigram_weight=3)
    #scores_wo_F5_phrase_weight = filter_and_rank_keywords(scores, stopwords, num_keywords=10)

    #FITUR-3 POS FILTER
    # Attaching POS Tags
    keywords_with_pos = attach_pos_tags(scores_w_F5, pos_tokens)
    # Filtering by Selected POS Tags
    selected_pos_tags = {'NN', 'NNP', 'VB', 'NP', 'VP', 'FW'}
    scores_w_F35 = filter_by_pos(keywords_with_pos, selected_pos_tags)

    #FITUR-4 WEIGHT WORD POSITION
    title = df["judul"]
    scores_w_F345 = adjust_scores_for_title(scores_w_F35, title)

    #FITUR-6 combined_TFIDF
    #words = detect_all_tokens(text)
    #scores_w_F3456 = compute_and_adjust_tfidf_scores(words, scores_w_F345)

    df_keyphrases = format_keyphrases_dataframe(scores_w_F345)

    predict_textrank = pd.concat([predict_textrank, df_keyphrases], ignore_index=True)
    print('Done')
    
predict_textrank.head(3)

In [38]:
# Load Word2Vec Model
#w2v_model_path = os.path.join(repo_root, "models/embedding_model/w2v_wiki_own_phrase_training_200.model")
#w2v_model, available_tokens = load_word2vec_model(w2v_model_path)

# Load Fasttext Model
#fasttext_model_path = os.path.join(repo_root, "models/embedding_model/fasttext_ina_200_with_phrases.bin") # sudah dipanggil diatas
#ft_model, available_tokens = load_fasttext_model(fasttext_model_path) # udah dipanggil diatas

predict_textrank = pd.DataFrame()
#for i in df.index:
for i in df.loc[2:5].index:    
    print('Processing index', i, end='...! ')
    # Processing Text
    text = df["text"][i]
    words = detect_all_tokens(text)
    # Postagging
    pos_tokens = postag_tokens(words)
    # Building Co-occurrence Matrix
    co_occurrence = build_co_occurrence_matrix(words)
    # Building Graph and Computing TextRank
    #scores = build_graph_and_compute_textrank(co_occurrence, w2v_model)
    scores = build_graph_and_compute_textrank(co_occurrence, ft_model)

    # FITUR-5 phrase_phrase_weight
    # Filtering and Ranking Keywords
    scores_w_F5 = filter_and_rank_keywords(scores, stopwords, num_keywords=10, bigram_weight=2, trigram_weight=3)
    #scores_wo_F5_phrase_weight = filter_and_rank_keywords(scores, stopwords, num_keywords=10)

    #FITUR-3 POS FILTER
    # Attaching POS Tags
    keywords_with_pos = attach_pos_tags(scores_w_F5, pos_tokens)
    # Filtering by Selected POS Tags
    selected_pos_tags = {'NN', 'NNP', 'VB', 'NP', 'VP', 'FW'}
    scores_w_F35 = filter_by_pos(keywords_with_pos, selected_pos_tags)

    #FITUR-4 WEIGHT WORD POSITION
    title = df["judul"]
    scores_w_F345 = adjust_scores_for_title(scores_w_F35, title)

    #FITUR-6 combined_TFIDF
    #words = detect_all_tokens(text)
    #scores_w_F3456 = compute_and_adjust_tfidf_scores(words, scores_w_F345)

    df_keyphrases = format_keyphrases_dataframe(scores_w_F345)

    
    # existing
    #df_keyphrases = pd.DataFrame(scores_w_F3456, columns=['Keyword', 'Score', 'pos'])
    #a = pd.DataFrame(df_keyphrases.Keyword).T.reset_index(drop=True)
    #a = a.reindex(columns=range(10), fill_value=0)
    #b = pd.DataFrame(df_keyphrases.Score).round(3).T.reset_index(drop=True)
    #b = b.reindex(columns=range(10), fill_value=0)
    #c = pd.DataFrame(df_keyphrases.pos).round(3).T.reset_index(drop=True)
    #c = c.reindex(columns=range(10), fill_value=0)
    #df_keyphrases = pd.concat([a, b, c], axis=1)

    # Check if there are missing columns and add them with zero values
    #missing_columns = 30 - df_keyphrases.shape[1]
    #for _ in range(missing_columns):
    #    df_keyphrases[df_keyphrases.shape[1]] = 0

    #df_keyphrases.columns = ['key_1', 'key_2','key_3', 'key_4', 'key_5','key_6', 'key_7', 'key_8','key_9','key_10',
    #                         'score_1', 'score_2','score_3','score_4', 'score_5','score_6','score_7', 'score_8','score_9','score_10',
    #                         'pos_1', 'pos_2','pos_3', 'pos_4', 'pos_5','pos_6', 'pos_7', 'pos_8','pos_9','pos_10']
    

    # Adding the formatted DataFrame to the cumulative DataFrame
    predict_textrank = pd.concat([predict_textrank, df_keyphrases], ignore_index=True)
    print('Done')
    
predict_textrank.head(3)

Processing index 2...! Done
Processing index 3...! 

Done
Processing index 4...! Done
Processing index 5...! Done


,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,...,pos_1,pos_2,pos_3,pos_4,pos_5,pos_6,pos_7,pos_8,pos_9,pos_10
0,change inquiry,kantor,ruangan,usulan,tidak mengalami,lapangan,appendix,lokasi,artikel,0,...,NP,NN,NN,NN,VP,NN,NN,NN,NN,0
1,counter part,counter,engineering,instrument,pengadaan,lead,control,menanggapi,konstruksi,0,...,NP,FW,FW,NN,NN,NN,FW,VP,NN,0
2,site survey,survey,lapangan,iwan,manager,hamzah,tanggai,construction,kunjungan,berkoordinasi,...,NP,FW,NN,NN,NN,NN,VP,FW,NN,VP


# EVALUATION

In [39]:
def check_similarity(preds_row, targets_row):
    result = []
    strict_accum = 0
    strict_prec = 0
    flex_accum = 0
    flex_prec = 0

    for pred in preds_row:
        pred = str(pred).lower().strip()
        state = 'no_match'

        for target in targets_row:
            target = str(target).lower().strip()

            if pred == target:
                state = 'full_match'
                flex_accum += 1
                strict_accum += 1
                break
            elif pred in target:
                state = 'partial_match'
                flex_accum += 1
                strict_accum += 0.5
                break
            # perlu ditambahkan jika pred ada terdiri dari bbrp kata, dan katanya berisisan dengan target, maka menjadi partial match
            '''
            elif target in pred:
                state = 'partial_match'
                flex_accum += 1
                strict_accum += 0.5
            '''
        result.append(state)

    len_targets_row = len(targets_row)
    len_preds_row = len(preds_row)

    if len_preds_row != 0:
        flex_recall = flex_accum / len_targets_row
        flex_prec = flex_accum / len_preds_row

    if len_targets_row != 0:
        strict_recall = strict_accum / len_targets_row
        strict_prec = strict_accum / len_preds_row

    result.extend([strict_recall, strict_prec, flex_recall, flex_prec])

    return result

def eval(predictions, targets, to_dataframe = False):
    result = []

    for i in range(len(predictions)):
      result.append(check_similarity(predictions[i], targets[i]))
    
    if to_dataframe:
        return pd.DataFrame(result)
    else:
        return result
    
def evaluate_prediction(predict_list, targets_list):
    '''
    fungsi untuk membandingkan hasil prediksi dengan gold truth
    dengan hasil exact match =1, partial match = 1, no match = 0, utk kemudian dihitung per row
    '''
    evaluation = eval(predict_list, targets_list, True).round(3)
    evaluation.columns = ['key_1', 'key_2','key_3','strict_recall', 'strict_prec', 'flex_recall','flex_prec']
    evaluation = evaluation[['key_1', 'key_2','key_3', 'flex_recall','flex_prec']]
    return evaluation

def summarize_evaluation(eval_method):
    '''
    fungsi untuk menghitung score metric recall, precision, dan F1
    dengan paramater flexible : exact match =1, partial match = 1, no match = 0
    hasil dalam bentuk dataframe summary
    '''
    recall = eval_method['flex_recall'].mean()
    prec = eval_method['flex_prec'].mean()
    f1 = 2 * (prec * recall) / (prec + recall)

    summary = pd.DataFrame({'metric': [recall, prec, f1]}, index=['recall', 'precision', 'F1'])
    summary = summary.round(3)
    return summary

In [ ]:
#from utils import eval
targets = df[["k1", "k2", "k3","k4", "k5", "k6","k7"]].values.tolist()
df_targets = pd.DataFrame(targets)

In [ ]:
# Evaluation TextRank top 10
predict_textrank_list_10 = predict_textrank[['key_1','key_2','key_3', 'key_4','key_5','key_6', 'key_7','key_8','key_9', 'key_10']].values.tolist()
eval_textrank_10 = eval(predict_textrank_list_10, targets, True).round(3)
eval_textrank_10.columns = ['key_1','key_2','key_3', 'key_4','key_5','key_6', 'key_7','key_8','key_9', 'key_10','strict_recall', 'strict_prec', 'flex_recall','flex_prec']
eval_textrank_10 = eval_textrank_10[['key_1','key_2','key_3', 'key_4','key_5','key_6', 'key_7','key_8','key_9', 'key_10', 'flex_recall','flex_prec']] # untuk menyederhanakan hasil evaluasi
#eval_textrank_10.head(3)

# Calculate TextRank Score, using flexible score : exact maatch =1, partial match = 1, no match = 0
textrank_recall_10 = eval_textrank_10['flex_recall'].mean()
textrank_prec_10 = eval_textrank_10['flex_prec'].mean()
textrank_f1_10 = 2 * (textrank_prec_10 * textrank_recall_10) / (textrank_prec_10 + textrank_recall_10)

# Create a DataFrame with the scores
summary_10 = pd.DataFrame({'textrank': [textrank_recall_10, textrank_prec_10, textrank_f1_10]}, index=['recall', 'precision', 'F1'])
summary_10 = summary_10.round(3)
#summary_10

In [ ]:
# Evaluation TextRank top 5
predict_textrank_list_5 = predict_textrank[['key_1','key_2','key_3', 'key_4','key_5']].values.tolist()
eval_textrank_5 = eval(predict_textrank_list_5, targets, True).round(3)
eval_textrank_5.columns = ['key_1','key_2','key_3', 'key_4','key_5','strict_recall', 'strict_prec', 'flex_recall','flex_prec']
eval_textrank_5 = eval_textrank_5[['key_1','key_2','key_3', 'key_4','key_5', 'flex_recall','flex_prec']] # untuk menyederhanakan hasil evaluasi
#eval_textrank_5.head(3)

# Calculate TextRank Score, using flexible score : exact maatch =1, partial match = 1, no match = 0
textrank_recall_5 = eval_textrank_5['flex_recall'].mean()
textrank_prec_5 = eval_textrank_5['flex_prec'].mean()
textrank_f1_5 = 2 * (textrank_prec_5 * textrank_recall_5) / (textrank_prec_5 + textrank_recall_5)

# Create a DataFrame with the scores
summary_5 = pd.DataFrame({'textrank': [textrank_recall_5, textrank_prec_5, textrank_f1_5]}, index=['recall', 'precision', 'F1'])
summary_5 = summary_5.round(3)
summary_5

,textrank
recall,0.174
precision,0.243
F1,0.202


In [ ]:
# Evaluation TextRank top 3
predict_textrank_list_3 = predict_textrank[['key_1','key_2','key_3']].values.tolist()
eval_textrank_3 = eval(predict_textrank_list_3, targets, True).round(3)
eval_textrank_3.columns = ['key_1', 'key_2','key_3','strict_recall', 'strict_prec', 'flex_recall','flex_prec']
eval_textrank_3 = eval_textrank_3[['key_1', 'key_2','key_3', 'flex_recall','flex_prec']] # untuk menyederhanakan hasil evaluasi
#eval_textrank_3.head(3)

# Calculate TextRank Score, using flexible score : exact maatch =1, partial match = 1, no match = 0
textrank_recall_3 = eval_textrank_3['flex_recall'].mean()
textrank_prec_3 = eval_textrank_3['flex_prec'].mean()
textrank_f1_3 = 2 * (textrank_prec_3 * textrank_recall_3) / (textrank_prec_3 + textrank_recall_3)

# Create a DataFrame with the scores
summary_3 = pd.DataFrame({'textrank': [textrank_recall_3, textrank_prec_3, textrank_f1_3]}, index=['recall', 'precision', 'F1'])
summary_3 = summary_3.round(3)
summary_3

,textrank
recall,0.125
precision,0.292
F1,0.175


In [ ]:
# Combine dataframe predict_textrank, df_targets and eval_textrank
predict_textrank_10 = pd.concat([predict_textrank, df_targets, eval_textrank_10], axis=1)
#predict_textrank_10.head(3)

# Combine dataframe predict_textrank, df_targets and eval_textrank
predict_textrank_5 = pd.concat([predict_textrank, df_targets, eval_textrank_5], axis=1)
#predict_textrank_5.head(3)

# Combine dataframe predict_textrank, df_targets and eval_textrank
predict_textrank_3 = pd.concat([predict_textrank, df_targets, eval_textrank_3], axis=1)
predict_textrank_3.head(3)

,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,...,2,3,4,5,6,key_1,key_2,key_3,flex_recall,flex_prec
0,personil penting,personil,organisasi,fase,tender,pengganti,usulan,email tersendiri,usulan personil penting,0,...,usulan,pengganti,NaN,NaN,NaN,no_match,no_match,no_match,0.000,0.000
1,document,acuan,template document,tiung,ctr,exhibit,processing,facilities,0,0,...,acuan,pengelolaan,dokumen,NaN,NaN,partial_match,full_match,full_match,0.429,1.000
2,kantor,ruangan,usulan,change inquiry,artikel,lokasi,accommodation,klarifikasi,0,0,...,lingkup kerja,akomodasi,services for company,exhibit a,NaN,partial_match,no_match,no_match,0.143,0.333


In [ ]:
import pandas as pd
from openpyxl import load_workbook

def write_excel(df, sheet_name, filename):
    """
    Writes the given dataframe to an excel file with the given filename and sheet name.
    If the sheet already exists in the file, the data in the sheet will be overwritten.
    """
    try:
        # Try to load the existing workbook
        with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
            if sheet_name in writer.book.sheetnames:
                # If sheet already exists, remove it
                sheet = writer.book[sheet_name]
                writer.book.remove(sheet)

            # Write the dataframe to the excel file
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    except FileNotFoundError:
        # If the file doesn't exist, create a new workbook
        with pd.ExcelWriter(filename, engine='openpyxl', mode='w') as writer:
            df.to_excel(writer, sheet_name=sheet_name, index=False)


In [ ]:
# Write predictions to excel file
#from utils import write_excel

sheet_name_10 = 'SE132a_adjusted_textrank_10'
sheet_name_5 = 'SE132a_adjusted_textrank_5'
sheet_name_3 = 'SE132a_adjusted_textrank_3'

output_file = 'SE132a_adjusted_textrank.xlsx'
write_excel(predict_textrank_10, sheet_name_10, output_file)
write_excel(predict_textrank_5, sheet_name_5, output_file)
write_excel(predict_textrank_3, sheet_name_3, output_file)

ADDITIONAL UTILITIES 
pilihan: 
- menunjukan gambar

In [ ]:
# Utilities
'''
def get_unique_tokens_pos(all_tokens, pos_model_path):
    """
    Get unique POS tags for tokens.
    """
    pos_model_path = os.path.join(repo_root, "notebooks/nlp-id_retraining/train_tuned.pkl")
    postagger = PosTag(pos_model_path)
    pos_tokens = []
    seen_tokens = set()
    
    for token in all_tokens:
        if token not in seen_tokens:
            seen_tokens.add(token)
            tokens_pos = postagger.get_phrase_tag(token)
            pos_tokens.append(tokens_pos)
    return pos_tokens
'''
'''
# Fungsi Visualisasi
def visualize_graph(G, labels):
    # Remove self-loops (edges that connect a node to itself)
    G.remove_edges_from(nx.selfloop_edges(G))

    fig = plt.figure(figsize=(12, 12))
    pos = nx.spring_layout(G, seed=42)
    #nx.draw(G, pos=pos, with_labels=False, font_weight="bold", node_size=5000, node_color='skyblue')
    nx.draw(G, pos=pos, with_labels=False, font_weight="bold")
    nx.draw_networkx_labels(G, pos, labels, font_size=12)
    plt.show()

'''

In [ ]:
'''
def generate_ngrams(words, n=2):
    """Generate ngrams from a list of words."""
    return [" ".join(gram) for gram in ngrams(words, n)]
'''